# Labelme to mask-batch

## Unzip the sample dataset

In [1]:
!unzip CFA87_Semantic_Seg_Labelme.zip >> /dev/null # Unzip the archive
!rm -rf CFA87_Semantic_Seg_Labelme.zip # Delete the archive

In [6]:
!pip install seedir emoji -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 112 kB 53.8 MB/s eta 0:00:01
     |████████████████████████████████| 590 kB 97.2 MB/s eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.0.1
    Uninstalling typing-extensions-4.0.1:
      Successfully uninstalled typing-extensions-4.0.1


## View dataset catalog structure

In [7]:
import seedir as sd
sd.seedir('CFA87_Semantic_Seg_Labelme', style='emoji', depthlimit=1)

📁 CFA87_Semantic_Seg_Labelme/
├─📁 images/
└─📁 labelme_jsons/


## Remove redundant files automatically generated by the system

### 查看待删除的多余文件

In [11]:
!find . -iname '__MACOSX'

In [12]:
!find . -iname '.DS_Store'

In [13]:
!find . -iname '.ipynb_checkpoints'

./.ipynb_checkpoints


### Remove redundant files

In [14]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

In [15]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

In [16]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

### Verify that redundant files have been deleted

In [17]:
!find . -iname '__MACOSX'

In [18]:
!find . -iname '.DS_Store'

In [19]:
!find . -iname '.ipynb_checkpoints'

## Go to the dataset catalog

In [20]:
import os
import json
import numpy as np
import cv2
import shutil

from tqdm import tqdm

## Data sets and category information

In [21]:
Dataset_Path = 'CFA87_Semantic_Seg_Labelme'

## Information about each category and the order in which the masks are drawn (in order from largest to smallest and coarse to fine)

In [22]:
# 0-background, start from 1
class_info = [
    {'label':'sandwich', 'type':'polygon', 'color':1},                    # polygon 多段线
    {'label':'fries', 'type':'polygon', 'color':2},
    {'label':'drink', 'type':'polygon', 'color':3}
]

## Single image labelme to mask function

In [23]:
def labelme2mask_single_img(img_path, labelme_json_path):
    '''
    Input the original image path and labelme annotation path, output the mask
    '''
    
    img_bgr = cv2.imread(img_path)
    img_mask = np.zeros(img_bgr.shape[:2]) # Create a blank image, 0-background
    
    with open(labelme_json_path, 'r', encoding='utf-8') as f:
        labelme = json.load(f)
        
    for one_class in class_info: # Iterate over each category in order
        for each in labelme['shapes']: # Iterate over all annotations, find annotations belonging to the current category
            if each['label'] == one_class['label']:
                if one_class['type'] == 'polygon': # polygon annotation

                    # Get the coordinates of the points
                    points = [np.array(each['points'], dtype=np.int32).reshape((-1, 1, 2))]

                    # Draw mask on the blank image (closed area)
                    img_mask = cv2.fillPoly(img_mask, points, color=one_class['color'])

                elif one_class['type'] == 'line' or one_class['type'] == 'linestrip': # line or linestrip annotation

                    # Get the coordinates of the points
                    points = [np.array(each['points'], dtype=np.int32).reshape((-1, 1, 2))]

                    # Draw mask on the blank image (non-closed area)
                    img_mask = cv2.polylines(img_mask, points, isClosed=False, color=one_class['color'], thickness=one_class['thickness']) 

                elif one_class['type'] == 'circle': # circle annotation

                    points = np.array(each['points'], dtype=np.int32)

                    center_x, center_y = points[0][0], points[0][1] # Center point coordinates

                    edge_x, edge_y = points[1][0], points[1][1]     # Edge point coordinates

                    radius = np.linalg.norm(np.array([center_x, center_y] - np.array([edge_x, edge_y]))).astype('uint32') # Radius

                    img_mask = cv2.circle(img_mask, (center_x, center_y), radius, one_class['color'], one_class['thickness'])

                else:
                    print('Unknown annotation type', one_class['type'])
                    
    return img_mask

## labelme to mask-batch

In [24]:
os.chdir(Dataset_Path)
os.mkdir('masks')
os.chdir('images')

In [25]:
for img_path in tqdm(os.listdir()):
    
    try:
    
        labelme_json_path = os.path.join('../', 'labelme_jsons', '.'.join(img_path.split('.')[:-1])+'.json')

        img_mask = labelme2mask_single_img(img_path, labelme_json_path)

        mask_path = img_path.split('.')[0] + '.png'

        cv2.imwrite(os.path.join('../','masks',mask_path), img_mask)
    
    except Exception as E:
        print(img_path, 'conversion failed', E)

100%|██████████| 73/73 [00:05<00:00, 13.57it/s]


## Converted masks are saved in the `masks` folder.

## Rename and delete folders

In [26]:
os.chdir('../')
shutil.move('images', 'img_dir')
shutil.move('masks', 'ann_dir')
!rm -rf labelme_jsons
os.chdir('../')

## Getting the final semantic segmentation dataset

## View dataset catalog structure

In [27]:
import seedir as sd
sd.seedir('CFA87_Semantic_Seg_Labelme', style='emoji', depthlimit=1)

📁 CFA87_Semantic_Seg_Labelme/
├─📁 ann_dir/
└─📁 img_dir/
